# LASER LOCK

In [1]:
# import libraries for devices
import time, datetime, csv
from pathlib import Path
from ipywidgets import interact

# import libraries for plotting etc
import matplotlib.pyplot as plt
import numpy as np 

%matplotlib inline

## 1. Connect to the RedPitaya

- The config file should already have all the modules opened up, otherwise open them and set all the values accordingly ( see labNotes...)
- Start ramping with the full range of 1 V such that you do not miss the signal

In [2]:
import pyrpl
p = pyrpl.Pyrpl(hostname="10.0.2.111", config="111.yml"); r = p.rp

INFO:pyrpl:All your PyRPL settings will be saved to the config file
    C:\Users\NaKaControl\pyrpl_user_dir\config\111.yml
If you would like to restart PyRPL with these settings, type "pyrpl.exe 111" in a windows terminal or 
    from pyrpl import Pyrpl
    p = Pyrpl('111')
in a python terminal.
INFO:pyrpl.redpitaya:Successfully connected to Redpitaya with hostname 10.0.2.111.


In [2]:
# shortcut to save data
def save_data(data, timestamp, device):
    full_path = Path(Path.home().as_posix()+'/Dropbox (CoQuMa)/LabNotes/NaKa/'+timestamp[:7]+'/'+timestamp[:10]+'/data')
    try :
        full_path.mkdir(parents=True, exist_ok=True)
    except FileExistsError :
        print('already exists!')
        full_path = Path(Path.cwd().as_posix()+'/data')
                        
    with open(str(full_path)+'\\' + device + '_'+timestamp[:10]+'.csv', 'a', newline='', encoding='UTF8') as f:
        writer = csv.writer(f)
        writer.writerow(data)

In [16]:
# just keep track of the lock...
moku = np.round(i.read_power_supply(1)['set_voltage'],3)
trigger = True

while True:
    now = datetime.datetime.now()
    timestamp = now.strftime("%Y-%m-%d %H:%M:%S")
    
    data = [ timestamp, np.round(r.pid0.ival,3), np.round(r.scope.voltage_in2,3), np.round(r.pid0.setpoint,3), moku ]
    
    save_data(data, timestamp, 'lock')
    print(data, end='\r')
    
    if (-0.99 >= r.scope.voltage_in1 or 0.99 <= r.scope.voltage_in1) and trigger :
        print('Out of lock!')
        alert(timestamp)
        trigger = False
        
    time.sleep(5)

ERROR:pyrpl.redpitaya_client:Error occured in reading attempt 0. Reconnecting at addr 0x40300100 to _reads value 1 by client 6


KeyboardInterrupt: 

## 2. Connect to the Moku:GO
- It may be, that the moku is already inizialized and won't respond. In that case, clear all connections first and then retry

In [3]:
from moku.instruments import Oscilloscope

## 3. Find a good offset for the ramping

In [4]:
i = Oscilloscope('10.0.2.110', force_connect=True)

# Here you can access the available_power_supplies function
i.available_power_supplies()

# We usually have to set the ramping off by about 2 V...
#_ = i.set_power_supply(1, enable=True, voltage=-1.75, current=0.15)
#_ = i.set_power_supply(2, enable=True, voltage=10.2, current=0)
# print(np.round(i.read_power_supply(1)['set_voltage'],3))
#i.relinquish_ownership()

[{'id': 1,
  'enabled': True,
  'current_range': [0, 0.15],
  'voltage_range': [-5, 5],
  'set_voltage': -1.75048828125,
  'set_current': 0.1495258935083881,
  'actual_voltage': -0.05550000000000033,
  'actual_current': -0.003452758789062482,
  'constant_current_mode': True,
  'constant_voltage_mode': False},
 {'id': 2,
  'enabled': True,
  'current_range': [0, 0.15],
  'voltage_range': [0, 16],
  'set_voltage': 10.2001953125,
  'set_current': 0.00364697301239976,
  'actual_voltage': 0.016,
  'actual_current': -9.1552734375e-05,
  'constant_current_mode': True,
  'constant_voltage_mode': False},
 {'id': 3,
  'enabled': False,
  'current_range': [0.07, 1],
  'voltage_range': [0.6, 5],
  'set_voltage': 0.6996367045587719,
  'set_current': 1,
  'actual_voltage': 0.7000000000000001,
  'actual_current': 0.000885009765625,
  'constant_current_mode': False,
  'constant_voltage_mode': True},
 {'id': 4,
  'enabled': False,
  'current_range': [0.07, 1],
  'voltage_range': [0.6, 5],
  'set_voltag

## 4. Zoom in unto the CO-resonance
- Decrease the amplitude of the ramping while adjusting the offset given by the Moku:GO such that the resonance stays in the center of the slope

## 5. Actual locking
- Turn on the fast ( 10 kHz ) modulation via the coil. Check for the iq output
- Set the setpoint of the PID, turn of the ramp and forward the PID output to the laser ( out 1 )

Enjoy your lock!

# APPENDIX

## Arm the Slackbot

In [5]:
import requests, json, os
from requests.exceptions import HTTPError
from urllib.parse import parse_qs
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

bot_token = os.getenv("SLACKBOT_TOKEN")

def alert(time):
    channel_id = "C02F1MWPLM7" # naka_laserlock channel
    url = "https://slack.com/api/chat.postMessage"
    payload = {
        "channel": "C02F1MWPLM7",
        #"text": "",
        "attachments": [{
            "text": ":fire: ALERT!!! \n The laser is out of lock! (" + time + ")",
            "color": "#ad1721",
            "attachment_type": "default",
            "actions": [{
                "name": "Take me there!",
                "text": "Take me there!",
                "type": "button",
                "style": "primary",
                "url": "https://hydra.kip.uni-heidelberg.de:8443/f21server/#",
                },
                {
                "name": "I don't care.",
                "text": "I don't care.",
                "type": "button",
                "style": "danger",
                "url": "https://media.tenor.co/images/dd7ce8ef69ccfe64aa3ad4d173390c66/tenor.gif"
                }]
        }]
    }
    headers = { 
        "Content-type" : "application/json",
        "Authorization" : "Bearer " + bot_token,
    }
    
    try:
        r = requests.post(url, json=payload, headers=headers)
        r.raise_for_status()
        print(json.dumps(json.loads(r.text), indent=4))
        #for ch in r.json()['channels']:
        #	print(ch['name'] + '---' + ch['id'])
    except HTTPError as http_err:
        print(str(http_err)) 
    except Exception as err:
        print(str(err))
    else:
        print('Message send!')

In [6]:
alert('lol')

HTTPSConnectionPool(host='slack.com', port=443): Max retries exceeded with url: /api/chat.postMessage (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C60960D2E0>: Failed to establish a new connection: [Errno 11002] getaddrinfo failed'))
